# Ecuación de Schrödinger dependiente del tiempo

Queremos resolver la ecuación de Schrödinger dependiente del tiempo en una dimensión espacial:
$$i\hbar\frac{\partial}{\partial t} \psi(x,t) = H  \psi(x,t)$$
dada una condición inicial $\psi(x,0)$ a tiempo cero. Como ejemplo vamos a considerar una función de onda de la forma
$$\psi(x,t=0) = \frac{1}{(2 \pi \sigma^2)^{1/4}} e^{(x-x_0)^2/4\sigma^2} e^{i k_0 x},$$
que corresponde a un paquete gaussiano centrado en $x_0$, de ancho $\sigma$ e impulso medio $p_0=\langle\psi|\hat{p}|\psi\rangle=\hbar k_0$, donde $\hat{p}=-i \hbar \tfrac{\partial}{\partial x}$ es el operador impulso.

El hamiltoniano que vamos a tratar tiene un término de energía cinética y un término de energía potencial:
$$
\begin{align}
H &= \frac{p^2}{2m}+V(x)\\&=- \frac{\hbar^2}{2m}\frac{\partial^2}{\partial x^2} + V(x).
\end{align}
$$

### Solución exacta para el paquete libre
La solución exacta para un paquete de onda gaussiano libre (sin potencial) en una dimensión está dada por (ver e.g. el [libro de Shankar](https://books.google.com.ar/books/about/Principles_of_Quantum_Mechanics.html?id=2zypV5EbKuIC&redir_esc=y)):

$$
\psi(x, t) = \frac{1}{\sqrt{\sqrt{2\pi\sigma^2} (1 + \frac{i \hbar t}{2 m \sigma^2})}} \exp\left(-\frac{(x-x_0-\frac{\hbar k_0 t}{m})^2}{4 \sigma^2 (1 + \frac{i \hbar t}{2 m \sigma^2})}\right) \exp\left(i k_0 x - i \frac{k_0^2 \hbar t}{2m}\right)
$$
Vemos que el paquete continúa siendo gaussiano a tiempo $t$, pero su posición se modifica linealmente con el tiempo (velocidad constante del centro) y su ancho aumenta (como $\sqrt{t}$ para tiempos largos $\to$ ¡difusión!).





## Método de Euler (FTCS)

Como el hamiltoniano no depende del tiempo explícitamente,
podemos integrar formalmente para obtener la función de onda a tiempo $t$ a partir de la función de onda a tiempo cero:
$$\psi(x,t)= e^{- i Ht/\hbar}\psi(x,0),$$
donde usamos la definición estándar de la exponencial de un operador (o matriz):
$$e^X = \sum_{n=0}^\infty {\frac{X^n}{n!}}.$$

Como $H$ es hermítico ($H=H^\dagger$), es fácil ver que el operador de evolución temporal $U(t)=e^{- i Ht/\hbar}$ es unitario $U(t)^\dagger U(t) =I$ y por lo tanto la norma de la función de onda se preserva en el tiempo.

Para calcular la evolución de manera numérica discretizamos el espacio y el tiempo. Definimos $x_j = j\Delta x$ y $t_n = n\Delta t$, con $\Delta x$ y $\Delta t$ los incrementos en el espacio y tiempo, respectivamente.
Usamos una notación más compacta para la función de onda discretizada: $\psi_{j}^{n} \equiv \psi(x_j, t_n)$. El número de puntos en la grilla espacial es $N=[L/\Delta x]$.

Si suponemos un intervalo de tiempo $\Delta t$ muy corto y hacemos un desarrollo de la exponencial a primer orden:
$$U(\Delta t)=e^{- i H \Delta t/\hbar} \sim 1 - i H \Delta t/\hbar,$$
Podemos usar esta expresión para obtener $\Psi^{n+1}$ a partir de $\Psi^{n}$:
$$\psi^{n+1} = U(\Delta t) \psi^{n}$$
Para eso necesitamos aplicar el hamiltoniano a la función de onda. Aproximando la derivada segunda espacial utilizando diferencias finitas:
$$
\frac{\partial^2 \psi_k^n}{\partial x^2} \approx \frac{\psi_{k-1}^n - 2\psi_k^n + \psi_{k+1}^n}{(\Delta x)^2}
$$
nos queda
$$
\psi_{k}^{n+1}=\psi_{k}^{n} -\frac{i\Delta t}{\hbar} (-\lambda \psi_{k+1} + (V_k + 2\lambda)\psi_{k} -\lambda \psi_{k-1}),
$$

Podemos representar a la función de onda discretizada como un vector:
$$
\begin{pmatrix}
\psi_0^{n}\\
\psi_1^{n}\\
\vdots\\
\psi_{N-1}^{n}
\end{pmatrix},
$$
por lo que un paso de evolución temporal se reduce a una multiplicación matriz vector:

$$
\begin{pmatrix}
\psi_0^{n+1}\\
\psi_1^{n+1}\\
\vdots\\
\psi_{N-1}^{n+1}
\end{pmatrix}= \left(I-\frac{iH\Delta t}{\hbar}\right)
\begin{pmatrix}
\psi_0^{n}\\
\psi_1^{n}\\
\vdots\\
\psi_{N-1}^{n}
\end{pmatrix}.
$$
En esta expresión $I$ es la matriz identidad y $H$ es la matriz hamiltoniana
$$
H=\begin{pmatrix}
V_0+2\lambda&-\lambda&0&0&0&\cdots&0\\
-\lambda&V_1+2\lambda&-\lambda&0&0&\cdots&0\\
0&-t\lambda&V_2+2\lambda&-\lambda&0&\cdots&0\\
0&0&-\lambda&V_3+2\lambda&-\lambda&\ddots&0\\
\vdots&\vdots&\ddots&\ddots&\ddots&\ddots&-\lambda\\
 0&0&0&0&\cdots&-\lambda&V_{N-1}+2\lambda
\end{pmatrix}
$$
donde $\lambda=\frac{\hbar^2}{2m\Delta x^2}$ y $V_j=V(j \Delta x)$.

Las condiciones de contorno que se usaron para construir $H$ son del tipo *pared dura* en $x=0$ y $x=L$. Esto es, la función de onda se anula fuera del dominio $[0,L]$.






### Ejemplo de la evolución usando el método de Euler

En el siguiente código implementamos el método de Euler para la evolución temporal de un paquete de onda gaussiano y comparamos con la solución exacta.

El problema con el desarrollo del operador de evolución temporal a primer orden es que el operador resultante ($1 - i H \Delta t/\hbar$) **no es unitario** y por lo tanto no preserva la norma de la función de onda.  Vamos a tener que normalizar la función de onda resultante. En el código lo hacemos luego de cada paso de integración.


Otro problema con este método es que hay que usar un paso de tiempo muy chico ($\Delta t\propto 1/\Delta x^2$) para evitar inestabilidades.





In [ ]:
# @title
# Método de Euler (no preserva la norma y no es estable...)
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
from IPython.display import HTML
import sys

# potencial externo
def potencial(x):
    V0 = 0.0
    if(x < 10 or x > 90):
        V0 = 0.0
    if(x > 45 and x < 55):
        V0 = 0.0
    return V0

# longitud del sistema
L = 100
# paso de discretización
dx = 0.1
# Paso de tiempo
# tiene que ser muy chico para que no se inestabilice
# dt/dx**2 <1 suponiendo otros parámetros O(1)
dt = 0.0001
# Tiempo total
tmax = 10
# Centro de la gaussiana
x0 = L / 4.0
# Ancho de la gaussiana
sigma = 2.0
# Vector de onda
k0 = 4.0
# Número de puntos de cuadrícula en el espacio convertido a entero
N = int(L / dx)
# Número de pasos de tiempo
Nt = int(tmax / dt)

# Masa de la partícula (1 para simplificar)
m = 1.0
# Constante de Planck reducida (1 para simplificar)
hbar = 1.0

# Cuadros en la animación
cuadros = 100
skip = Nt/cuadros

psi = np.zeros(N, dtype=complex)
aux = np.zeros(N, dtype=complex)

hop = 1.0/(2* dx**2)

H = np.zeros((N,N), dtype=complex)

for i in range(N):
  H[i,i] = potencial(i*dx) + 2*hop

for i in range(N-1):
  H[i  ,i+1] = -hop
  H[i+1,i  ] = -hop

def psi_exact(x, t):
    sigma_t = sigma * np.sqrt(1 + 1j*hbar * t / (2 * m * sigma**2) )
    gauss = np.exp(-(x-x0-hbar * k0 * t / m)**2/ (4 * sigma_t**2))
    phase = np.exp(1j * k0 * x - 1j* k0**2 * hbar* t/ (2*m) )
    return  gauss * phase / (sigma_t * np.sqrt(np.sqrt(2 * np.pi)/sigma))

# Inicialización de la función de onda, normalizada
for i in range(N):
    psi[i] = psi_exact(i * dx, 0)

# Inicialización de la función de onda exacta, normalizada
psi_exact_init = psi_exact(np.linspace(0, L, N), 0)

# Para almacenar la función de onda a diferentes tiempos
evolution = []

# Para almacenar la función de onda exacta a diferentes tiempos
evolution_exact = []

# Evolución de la función de onda
for t in range(Nt):
    psi = psi - (1j * dt) * H.dot(psi)
    # Normalizamos
    psi = psi/np.sqrt(dx * np.dot(psi.conjugate(),psi))
    # Saltea algunos tiempos
    if(t%skip == 0):
        evolution.append(np.abs(psi)**2)
        evolution_exact.append(np.abs(psi_exact(np.linspace(0, L, N), t * dt))**2)

fig = plt.figure(figsize=(8,6))
ax = plt.axes(xlim=(0, L), ylim=(0, 0.4))

line, = ax.plot([], [], lw=2)

# Fondo de la imagen
def init():
    line.set_data([], [])
    return line,

# Cuadros de la animacion

def aframe(i):
    ax.clear()
    ax.set_xlim(0, L)
    ax.set_ylim(0, 0.4)
    ax.set_xlabel('x')
    ax.set_ylabel('$|\\psi|^2$')
    x = np.linspace(0, L, N)
    y = evolution[i]
    y_exact = evolution_exact[i]
    #line.set_data(x, y)
    line, = ax.plot(x, y, lw=2, label = "Euler")
    ax.plot(x, y_exact, color='red', linestyle='dashed', label = "Analítica")
    ax.legend()
    return line,

# Animador
anim = animation.FuncAnimation(fig, aframe, init_func=init,
                               frames=cuadros, interval=100, blit=True)

# Cierra la figura para que no haga un plot estático abajo de la animación
plt.close(fig)

# Animación
HTML(anim.to_jshtml())



Output hidden; open in https://colab.research.google.com to view.

##Método de Crank-Nicolson

El método de Crank-Nicolson es una alternativa que, a diferencia del método de Euler, es estable y conserva la norma de la función de onda.


La descomposición de Cayley es una forma de aproximar el operador de evolución temporal utilizando una expresión que preserva la unitaridad y además es una aproximación de $\mathcal{O}(\Delta t^2)$ de la exponencial:
$$
U(\Delta t) \approx \frac{1 - \frac{iH\Delta t}{2\hbar}}{1 + \frac{iH\Delta t}{2\hbar}}.
$$
Usando que el operador $H$ es hermítico
$$
U^\dagger(\Delta t) \approx \frac{1 + \frac{iH\Delta t}{2\hbar}}{1 - \frac{iH\Delta t}{2\hbar}},
$$
por lo que $U^\dagger U=I$ es unitario.

Además
$$
U(\Delta t) \approx \frac{1 - \frac{iH\Delta t}{2\hbar}}{1 + \frac{iH\Delta t}{2\hbar}}\sim \left(1 - \frac{iH\Delta t}{2\hbar}\right)\left[1 - \frac{iH\Delta t}{2\hbar}+\left(\frac{iH\Delta t}{2\hbar}\right)^2\right]\sim 1-\frac{iH\Delta t}{\hbar}+\frac{1}{2}\left(\frac{iH\Delta t}{\hbar}\right)^2
$$
es correcto hasta segundo orden.

Usando la expresión de Cayley para $U$ nos queda:
$$\left( 1 + i H \Delta t/2\hbar\right)\psi^{n+1} =  \left( 1 - i H \Delta t/2\hbar\right)\psi^{n}$$

 Este es un método implícito de la familia de métodos de [Crank-Nicolson](https://doi.org/10.1017/S0305004100023197). Al discretizar el espacio en $N$ puntos, nos van a quedar una matriz de $N\times N$ por invertir (eso es $\mathcal{O}(N^3)$ operaciones) y vamos a tener que aplicar la matriz inversa al término de la derecha (eso es $\mathcal{O}(N^2)$ operaciones). La matriz de la derecha es tridiagonal, así que aplicarla es $\mathcal{O}(N)$.



### Método de Watanabe y Tsukada

Usando un par de trucos descritos en esta referencia:
*  *Fast and stable method for simulating quantum electron dynamics*, Naoki Watanabe, Masaru Tsukada, Physical Review E. **62**, 2914, (2000)

se puede bajar fuertemente el costo computacional para sistemas en los que tenemos un hamiltoniano de la forma:

$$\mathcal{H}= - \frac{\hbar^2}{2m}\frac{\partial^2}{\partial x^2} + V(x)$$

Tenemos entonces:
$$
\begin{align}
\psi(x,t + \Delta t)&= e^{- i \mathcal{H} \Delta t/\hbar}\psi(x,t)\\
&= e^{ \frac{i\Delta t \hbar}{2m} \frac{\partial^2}{\partial x^2} -i \Delta t V(x)/\hbar}\psi(x,t).\\
\end{align}
$$
En esta expresión aparece la exponencial de la suma de dos operadores:
$$
e^{\Delta t \hat{K} + \Delta t \hat{V}},
$$
con $\hat{K}= \frac{i\hbar}{2m} \frac{\partial^2}{\partial x^2}$ y $\hat{V}=i V(x)/\hbar$.

Como $\hat{K}$ y $\hat{V}$  en general no conmutan, no podemos escribir la exponencial de la suma como el producto de exponenciales (como haríamos con números).

Pero, se puede [demostrar](https://es.wikipedia.org/wiki/F%C3%B3rmula_de_Baker-Campbell-Hausdorff) que (fórmula de Zassenhaus)

$$
e^{\Delta t( \hat{K}+\hat{V})}= e^{\Delta t\hat{K}}~ e^{\Delta t\hat{V}}
e^{-\frac{\Delta t^2}{2} [\hat{K},\hat{V}]} ~
e^{\frac{\Delta t^3}{6}(2[\hat{V},[\hat{K},\hat{V}]]+ [\hat{K},[\hat{K},\hat{V}] )} \cdots
$$
lo que nos premite hacer aproximaciones cuando $\Delta t\to 0$.

Suzuki encontró una serie de aproximaciones que evitan tener que hacer el cálculo de conmutadores. La aproximación correcta hasta segundo orden en $\Delta t$ es
$$
e^{\Delta t( \hat{K}+\hat{V})}\approx e^{\Delta t\frac{\hat{V}}{2}}~e^{\Delta t\hat{K}}~ e^{\Delta t\frac{\hat{V}}{2}} +\mathcal{O}(\Delta t^3)
$$
que se puede comprobar haciendo el desarrollo en serie de cada una de las exponenciales hasta segundo orden (**cada una de ellas es un operador unitario**).

Una ventaja de esta expresión es que en el espacio real el operador $\hat{V}$ es diagonal. Si lo pensamos en el dominio discretizado es una matriz diagonal. La exponencial de una matriz diagonal es otra matriz diagonal en la que que hacemos la exponencial de las componentes.
Para
$$
\hat{V}=\begin{pmatrix}
\tfrac{i V_0}{\hbar}&0&0&\cdots&0\\
0&\tfrac{i V_1}{\hbar}&0&\cdots&0\\
\vdots&\vdots&\vdots&\ddots&0\\
0&0&\cdots&0&\tfrac{i V_{N-1}}{\hbar}
\end{pmatrix}
$$
tenemos
$$
e^\frac{\Delta t \hat{V}}{2}=\begin{pmatrix}
e^{\tfrac{i \Delta t V_0}{2\hbar}}&0&0&\cdots&0\\
0&e^{\tfrac{ i \Delta t V_1}{2\hbar}}&0&\cdots&0\\
\vdots&\vdots&\vdots&\ddots&0\\
0&0&\cdots&0&e^{\tfrac{i \Delta t V_{N-1}}{2\hbar}}
\end{pmatrix}
$$

El operador de energía cinética es más dificil de tratar pero hay (al menos) dos opciones eficientes de hacerlo. Una es trabajar en el espacio de Fourier donde la energía cinética es diagonal. Eso requiere ir y volver del espacio real al de Fourier para ir aplicando alternadamente los operadores exponenciales (**hay que usar transformada rápida de Fourier (FFT)**). Esto se basa en que el operador de energía cinética es diagonal en el espacio transformado. La transformada de Fourier es una transformación unitaria que nos lleva de una base de funciones de onda localizadas en los sitios a otra de ondas planas.

La otra opción eficiente es usar la aproximación de Cayley para el operador de energía cinética:
$$
e^{\Delta t \hat{K}}\approx \frac{1+\Delta t \hat{K}/2}{1-\Delta t \hat{K}/2}
$$
Si se lo aplicamos a una función $\phi(x)$ nos da otra función $\chi(x)$:
$$
\chi(x,t) = e^{\Delta t \hat{K}}\phi(x,t)
$$
por lo que nos queda
$$
(1-\Delta t \hat{K}/2) \chi(x)= (1+\Delta t \hat{K}/2)\phi(x)
$$
que es lo mismo que
$$
\left[1-\Delta t \frac{i\hbar}{4m}\frac{\partial^2}{\partial x^2}\right] \chi(x)= \left[1+\Delta t \frac{i\hbar}{4m}\frac{\partial^2}{\partial x^2}\right]\phi(x)
$$

Discretizando la coordenada $x$, $\phi_k=\phi(k\Delta x)$ y usando la aproximación centrada para la derivada segunda,
$$
\frac{\partial^2 \psi_k}{\partial x^2} \approx \frac{\psi_{k-1}^n - 2\psi_k^n + \psi_{k+1}^n}{(\Delta x)^2}
$$
nos queda

$$
-\frac{\Delta t}{\Delta x^2} \frac{i\hbar}{4m}\chi_{k-1}+ \left(1+\frac{\Delta t}{\Delta x^2} \frac{i\hbar}{2m}\right) \chi_k-\frac{\Delta t}{\Delta x^2} \frac{i\hbar}{4m}\chi_{k+1}=
\frac{\Delta t}{\Delta x^2} \frac{i\hbar}{4m}\phi_{k-1}+ \left(1-\frac{\Delta t}{\Delta x^2} \frac{i\hbar}{2m}\right) \phi_k + \frac{\Delta t}{\Delta x^2} \frac{i\hbar}{4m}\phi_{k+1}
$$

Ahora dividimos la expresión por $\frac{i \hbar \Delta t}{4 m\Delta x^2}$ y nos queda
$$
-\chi_{k-1}+ A \chi_k-\chi_{k+1}=
\phi_{k-1}+ B \phi_k + \phi_{k+1}
$$
donde
$$
A=-4i\frac{m \Delta x^2}{\hbar \Delta t} + 2
$$
y
$$
B=-4i\frac{m\Delta x^2}{\hbar \Delta t} - 2.
$$

Podemos escribir estas ecuaciones en forma matricial. Por ejemplo para 4 sitios obtenemos:

$$
\begin{pmatrix}
A&-1&0&0\\
-1&A&-1&0\\
0&-1&A&-1\\
0&0&-1&A
\end{pmatrix}
\begin{pmatrix}\chi_0\\\chi_1\\\chi_2\\\chi_3\end{pmatrix}=
\begin{pmatrix}
B&1&0&0\\
1&B&1&0\\
0&1&B&1\\
0&0&1&B
\end{pmatrix}
\begin{pmatrix}\phi_0\\\phi_1\\\phi_2\\\phi_3\end{pmatrix}
$$
donde usamos condiciones de contorno de pared dura (las funciones se anulan en los bordes porque ponemos una energía potencial infinita fuera del dominio).

Aplicamos la matriz al vector en el término de la derecha para definir el vector auxiliar $\phi'$:
$$
\begin{pmatrix}\phi'_0\\\phi'_1\\\phi'_2\\\phi'_3\end{pmatrix}\equiv
\begin{pmatrix}
B&1&0&0\\
1&B&1&0\\
0&1&B&1\\
0&0&1&B
\end{pmatrix}
\begin{pmatrix}\phi_0\\\phi_1\\\phi_2\\\phi_3\end{pmatrix}
=\begin{pmatrix}B\phi_0 + \phi_1\\\phi_0+ B \phi_1+\phi_2\\\phi_1+B\phi_2+\phi_3\\\phi2+B\phi_3\end{pmatrix}
$$

La matriz en el término de la izquierda se puede invertir de manera eficiente porque su descomposición $LU$ es analítica (L y U se refieren a matrices triangulares inferior y superior respectivamente):

$$
\begin{pmatrix}
A&-1&0&0\\
-1&A&-1&0\\
0&-1&A&-1\\
0&0&-1&A
\end{pmatrix}
=
\begin{pmatrix}
u_0^{-1}&0&0&0\\
-1&u_1^{-1}&0&0\\
0&-1&u_2^{-1}&0\\
0&0&-1&u_3^{-1}
\end{pmatrix}
\begin{pmatrix}
1&-u_0&0&0\\
0&1&-u_1&0\\
0&0&1&-u_2\\
0&0&0&1
\end{pmatrix}
$$

con
$$
u_i=1/(A-u_{i-1}),\quad u_{-1}=0,
$$

La descomposición LU permite resolver el sistema de ecuaciones haciendo retrosustitución en dos etapas (con la matriz L y con la U). Definimos otro vector auxiliar $y$:

$$
\begin{pmatrix}
u_0^{-1}&0&0&0\\
-1&u_1^{-1}&0&0\\
0&-1&u_2^{-1}&0\\
0&0&-1&u_3^{-1}
\end{pmatrix}
\underbrace{\begin{pmatrix}
1&-u_0&0&0\\
0&1&-u_1&0\\
0&0&1&-u_2\\
0&0&0&1
\end{pmatrix}
\begin{pmatrix}\chi_0\\\chi_1\\\chi_2\\\chi_3\end{pmatrix}}_{\begin{pmatrix}y_0\\y_1\\y_2\\y_3\end{pmatrix}}
=\begin{pmatrix}\phi'_0\\\phi'_1\\\phi'_2\\\phi'_3\end{pmatrix}$$

Podemos obtener las componentes del vector auxiliar $y$:
\begin{align}
y_0 u_0^{-1}&=  \phi'_0\\
-y_0 + u_1^{-1} y_1&= \phi'_1\\
-y_1 + u_2^{-1} y_2&= \phi'_2\\
-y_2 + u_3^{-1} y_3&= \phi'_3
\end{align}
resolviendo estas ecuaciones de arriba hacia abajo. Una vez obtenidos los $y_k$ podemos obtener los $\chi_k$ de

$$
\begin{pmatrix}
1&-u_0&0&0\\
0&1&-u_1&0\\
0&0&1&-u_2\\
0&0&0&1
\end{pmatrix}
\begin{pmatrix}\chi_0\\\chi_1\\\chi_2\\\chi_3\end{pmatrix}=\begin{pmatrix}y_0\\y_1\\y_2\\y_3\end{pmatrix}
$$
resolviendo a partir de la última ecuación (última fila de la matriz) hasta la primera.

Un paso de integración se reduce a aplicar la exponencial para el potencial dividido $2$, aplicar la exponencial del operador energía cinética y aplicar nuevamente *medio* potencial.
$$
\psi^{n+1}= e^{\Delta t\frac{\hat{V}}{2}}~e^{\Delta t\hat{K}}~ e^{\Delta t\frac{\hat{V}}{2}} \psi_n$$

El método se puede aplicar en dos o tres dimensiones espaciales, usando que $p_x$, $p_y$ y $p_z$ conmutan. El operador de energía cinética total se descompone como el producto de operadores para cada componente.


### Paquete gaussiano libre
El código de abajo aplica el método de Watanabe y Tsukada para un paquete gaussiano libre con impulso medio finito ($m=1$, $\hbar=1$). El potencial está en cero pero se puede modificar. El código tiene como condición de contorno que la función de onda se anule en los bordes.

La condición inicial es
$$
\psi(x,0)= \left(\frac{1}{\sigma \sqrt{2 \pi}}\right)^{\tfrac{1}{2}} e^{-\left(\frac{x-x_0}{2\sigma}\right)^2 + ik_0 x}
$$

La [solución analítica está en Wikipedia](https://en.wikipedia.org/wiki/Wave_packet#Dispersive)

In [ ]:
# @title
# Método de Crank-Nicolson
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
from IPython.display import HTML
import sys

# potencial externo
def potencial(x):
    V0 = 0.0
    if(x < 10 or x > 90):
        V0 = 0.0
    if(x > 45 and x < 55):
        V0 = 0.0
    return V0

# longitud del sistema
L = 100.0
# paso de tiempo
dt = 0.01
# tiempo total
tmax = 10
# centro de la gaussiana
x0 = L / 4.0
# ancho de la gaussiana
sigma = 2.0
# vector de onda
k0 = 4.0
# Número de puntos de cuadrícula en el espacio convertido a entero
N = 1000
# Número de pasos de tiempo
Nt = int(tmax / dt)

# Cuadros en la animación
cuadros = 100
skip = Nt/cuadros

# paso de discretización
dx = L/(N-1)

# función de onda
psi = np.zeros(N, dtype=complex)
aux = np.zeros(N, dtype=complex)

# Parámetros auxiliares para la inversión del operador de energía cinética
bi = np.zeros(N, dtype=complex)
uop = np.zeros(N, dtype=complex)

A = 2.0 - 4j * (dx ** 2) / (dt)
B = -2.0 - 4j * (dx ** 2) / (dt)

uop[0] = 1.0 / A
for i in range(1, N):
    uop[i] = 1.0 / (A - uop[i - 1])

# Definimos el potencial
V = np.zeros(N)
for i in range(1, N):
    V[i] = potencial(i * dx)


# Inicializamos de la función de onda, normalizada
#for i in range(N):
#    psi[i] = (np.exp(-(i * dx - x0) ** 2 / (2 *sigma) ** 2)) * np.exp(1j * k0 * i * dx)/ np.sqrt(sigma * np.sqrt(2*np.pi))

def psi_exact(x, t):
    sigma_t = sigma * np.sqrt(1 + 1j*hbar * t / (2 * m * sigma**2) )
    gauss = np.exp(-(x-x0-hbar * k0 * t / m)**2/ (4 * sigma_t**2))
    phase = np.exp(1j * k0 * x - 1j* k0**2 * hbar* t/ (2*m) )
    return  gauss * phase / (sigma_t * np.sqrt(np.sqrt(2 * np.pi)/sigma))

# Inicialización de la función de onda, normalizada
for i in range(N):
    psi[i] = psi_exact(i * dx, 0)

# Inicialización de la función de onda exacta, normalizada
psi_exact_init = psi_exact(np.linspace(0, L, N), 0)

# Normalizamos de nuevo por las dudas...
psi = psi/np.sqrt(dx * np.dot(psi.conjugate(),psi))

# Aplicamo el término de energía potencial. Es diagonal, así que equivale a multiplicar por una fase
def aplica_medioV(fdo, V):
    for j in range(N):
        fdo[j] = fdo[j] * np.exp(-1.0j * dt * V[j] / 2.0)
    return fdo

# para almacenar la función de onda a diferentes tiempos y porder hacer una animación
evolution = []

# Para almacenar la función de onda exacta a diferentes tiempos
evolution_exact = []

# evolución de la función de onda
for t in range(Nt):

    psi = aplica_medioV(psi, V)

    """ Aplicamos la parte de energía cinética usando la descomposición LU"""
    # En el borde
    bi[0] = B * psi[0] + psi[1]
    bi[N - 1] = psi[N - 2] + B * psi[N - 1]
    # en el resto de los puntos
    for j in range(1, N - 1):
        bi[j] = psi[j - 1] + B * psi[j] + psi[j + 1]

    # propagación hacia adelante
    aux[0] = uop[0] * bi[0]
    for j in range(1, N):
        aux[j] = uop[j] * (bi[j] + aux[j - 1])

    # propagación hacia atrás
    psi[N - 1] = aux[N - 1]
    for j in reversed(range(N-1)):
      psi[j] = aux[j] + uop[j] * psi[j+1]

    psi = aplica_medioV(psi,V)

    #psi = psi/np.sqrt(dx * np.dot(psi.conjugate(),psi))

    #saltea algunos tiempos
    if(t%skip == 0):
      evolution.append(np.abs(psi)**2)
      evolution_exact.append(np.abs(psi_exact(np.linspace(0, L, N), t * dt))**2)



# El resto es para hacer la animación
fig = plt.figure(figsize=(8,6))
ax = plt.axes(xlim=(0, L), ylim=(0, 0.4))
line, = ax.plot([], [], lw=2)

# primer frame
def init():
    line.set_data([], [])
    return line,

# Cada cuadro corresponde a un tiempo
def aframe(i):
    ax.clear()
    ax.set_xlim(0, L)
    ax.set_ylim(0, 0.4)
    x = np.linspace(0, L, N)
    y = evolution[i]
    ax.set_xlabel('x')
    ax.set_ylabel('$|\\psi|^2$')
    y_exact = evolution_exact[i]
    line, = ax.plot(x, y, lw=2, label = "Crank-Nicolson")
    ax.plot(x, y_exact, color='red', linestyle='dashed', label="Analítica")
    ax.legend()
    return line,

# Animación.  Con blit=True solo reescribe lo que cambia
anim = animation.FuncAnimation(fig, aframe, init_func=init,
                               frames=cuadros, interval=100, blit=True)

#cierra la figura para que no haga un plot estático abajo de la animación
plt.close(fig)
#animación
HTML(anim.to_jshtml())



Output hidden; open in https://colab.research.google.com to view.

In [5]:
# @title
# Método de Crank-Nicolson
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
from IPython.display import HTML
import sys

# Parámetros de la barrera de potencial
ancho_barrera = 10
alto_barrera= 4

# potencial externo
def potencial(x):
    V0 = 0.0
    if(x > L/2.0 - ancho_barrera/2.0 and x < L/2.0 + ancho_barrera/2.0):
        V0 = alto_barrera * (x - L/2.0 + ancho_barrera/2.0)/ancho_barrera
    return V0


hbar = 1.0
m = 1.0
# longitud del sistema
L = 100.0
# paso de tiempo
dt = 0.01
# tiempo total
tmax = 20
# centro de la gaussiana
x0 = L / 4.0
# ancho de la gaussiana
sigma = 2.0
# vector de onda para que choque de manera resonante con la barrera
k0 = np.sqrt(np.pi**2/ancho_barrera**2 + 2.0*alto_barrera)
#k0 = np.sqrt(0.1 + 2.0*alto_barrera)

# energías pozo infinito: pi**2/2 ancho**2
# energía cinética: k0**2/2

# Número de puntos en el espacio
N = 1000
# Número de pasos de tiempo
Nt = int(tmax / dt)

# Cuadros en la animación
cuadros = 100
skip = Nt/cuadros

# paso de discretización
dx = L/(N-1)

# función de onda
psi = np.zeros(N, dtype=complex)
aux = np.zeros(N, dtype=complex)

# Parámetros auxiliares para la inversión del operador de energía cinética
bi = np.zeros(N, dtype=complex)
uop = np.zeros(N, dtype=complex)

A = 2.0 - 4j * (dx ** 2) / (dt)
B = -2.0 - 4j * (dx ** 2) / (dt)

uop[0] = 1.0 / A
for i in range(1, N):
    uop[i] = 1.0 / (A - uop[i - 1])

# Definimos el potencial
V = np.zeros(N)
for i in range(1, N):
    V[i] = potencial(i * dx)


# Inicializamos de la función de onda, normalizada
#for i in range(N):
#    psi[i] = (np.exp(-(i * dx - x0) ** 2 / (2 *sigma) ** 2)) * np.exp(1j * k0 * i * dx)/ np.sqrt(sigma * np.sqrt(2*np.pi))

def psi_exact(x, t):
    sigma_t = sigma * np.sqrt(1 + 1j*hbar * t / (2 * m * sigma**2) )
    gauss = np.exp(-(x-x0-hbar * k0 * t / m)**2/ (4 * sigma_t**2))
    phase = np.exp(1j * k0 * x - 1j* k0**2 * hbar* t/ (2*m) )
    return  gauss * phase / (sigma_t * np.sqrt(np.sqrt(2 * np.pi)/sigma))

# Inicialización de la función de onda, normalizada
for i in range(N):
    psi[i] = psi_exact(i * dx, 0)

# Inicialización de la función de onda exacta, normalizada
psi_exact_init = psi_exact(np.linspace(0, L, N), 0)

# Normalizamos de nuevo por las dudas...
psi = psi/np.sqrt(dx * np.dot(psi.conjugate(),psi))

# Aplicamos el término de energía potencial. Es diagonal, así que equivale a multiplicar por una fase
def aplica_medioV(fdo, V):
    for j in range(N):
        fdo[j] = fdo[j] * np.exp(-1.0j * dt * V[j] / 2.0)
    return fdo

# para almacenar la función de onda a diferentes tiempos y porder hacer una animación
evolution = []

# Para almacenar la función de onda exacta a diferentes tiempos
evolution_exact = []

# evolución de la función de onda
for t in range(Nt):

    psi = aplica_medioV(psi, V)

    """ Aplicamos la parte de energía cinética usando la descomposición LU"""
    # En el borde
    bi[0] = B * psi[0] + psi[1]
    bi[N - 1] = psi[N - 2] + B * psi[N - 1]
    # en el resto de los puntos
    for j in range(1, N - 1):
        bi[j] = psi[j - 1] + B * psi[j] + psi[j + 1]

    # propagación hacia adelante
    aux[0] = uop[0] * bi[0]
    for j in range(1, N):
        aux[j] = uop[j] * (bi[j] + aux[j - 1])

    # propagación hacia atrás
    psi[N - 1] = aux[N - 1]
    for j in reversed(range(N-1)):
      psi[j] = aux[j] + uop[j] * psi[j+1]

    psi = aplica_medioV(psi,V)

    #psi = psi/np.sqrt(dx * np.dot(psi.conjugate(),psi))

    #saltea algunos tiempos
    if(t%skip == 0):
      evolution.append(np.abs(psi)**2)
      evolution_exact.append(np.abs(psi_exact(np.linspace(0, L, N), t * dt))**2)

# El resto es para hacer la animación
fig = plt.figure(figsize=(8,6))
ax = plt.axes(xlim=(0, L), ylim=(0, 0.4))
line, = ax.plot([], [], lw=2)

# primer frame
def init():
    line.set_data([], [])
    return line,

# Cada cuadro corresponde a un tiempo
def aframe(i):
    ax.clear()
    ax.set_xlim(0, L)
    ax.set_ylim(0, 0.4)
    x = np.linspace(0, L, N)
    y = evolution[i]
    ax.set_xlabel('x')
    ax.set_ylabel('$|\\psi|^2$')
    y_exact = evolution_exact[i]
    line, = ax.plot(x, y, lw=2, label = "Crank-Nicolson")
    ax.plot(x, y_exact, color='red', linestyle='dashed', label="Analítica libre")
    ax.plot(x,V/20, color='green', label="potencial reescaleado")
    ax.legend()
    return line,

# Animación.  Con blit=True solo reescribe lo que cambia
anim = animation.FuncAnimation(fig, aframe, init_func=init,
                               frames=cuadros, interval=100, blit=True)

#cierra la figura para que no haga un plot estático abajo de la animación
plt.close(fig)
#animación
HTML(anim.to_jshtml())



Output hidden; open in https://colab.research.google.com to view.

In [6]:
# @title
# Método de Crank-Nicolson
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
from IPython.display import HTML
import sys

# Parámetros de la barrera de potencial
ancho_barrera = 10
alto_barrera= 4

# potencial externo
def potencial(x):
    V0 = 0.0
    if(x > L/2.0 - ancho_barrera/2.0 and x < L/2.0 + ancho_barrera/2.0):
        V0 = -alto_barrera * (x - L/2.0 - ancho_barrera/2.0)/ancho_barrera
    return V0


hbar = 1.0
m = 1.0
# longitud del sistema
L = 100.0
# paso de tiempo
dt = 0.01
# tiempo total
tmax = 20
# centro de la gaussiana
x0 = L / 4.0
# ancho de la gaussiana
sigma = 2.0
# vector de onda para que choque de manera resonante con la barrera
k0 = np.sqrt(np.pi**2/ancho_barrera**2 + 2.0*alto_barrera)
#k0 = np.sqrt(0.1 + 2.0*alto_barrera)

# energías pozo infinito: pi**2/2 ancho**2
# energía cinética: k0**2/2

# Número de puntos en el espacio
N = 1000
# Número de pasos de tiempo
Nt = int(tmax / dt)

# Cuadros en la animación
cuadros = 100
skip = Nt/cuadros

# paso de discretización
dx = L/(N-1)

# función de onda
psi = np.zeros(N, dtype=complex)
aux = np.zeros(N, dtype=complex)

# Parámetros auxiliares para la inversión del operador de energía cinética
bi = np.zeros(N, dtype=complex)
uop = np.zeros(N, dtype=complex)

A = 2.0 - 4j * (dx ** 2) / (dt)
B = -2.0 - 4j * (dx ** 2) / (dt)

uop[0] = 1.0 / A
for i in range(1, N):
    uop[i] = 1.0 / (A - uop[i - 1])

# Definimos el potencial
V = np.zeros(N)
for i in range(1, N):
    V[i] = potencial(i * dx)


# Inicializamos de la función de onda, normalizada
#for i in range(N):
#    psi[i] = (np.exp(-(i * dx - x0) ** 2 / (2 *sigma) ** 2)) * np.exp(1j * k0 * i * dx)/ np.sqrt(sigma * np.sqrt(2*np.pi))

def psi_exact(x, t):
    sigma_t = sigma * np.sqrt(1 + 1j*hbar * t / (2 * m * sigma**2) )
    gauss = np.exp(-(x-x0-hbar * k0 * t / m)**2/ (4 * sigma_t**2))
    phase = np.exp(1j * k0 * x - 1j* k0**2 * hbar* t/ (2*m) )
    return  gauss * phase / (sigma_t * np.sqrt(np.sqrt(2 * np.pi)/sigma))

# Inicialización de la función de onda, normalizada
for i in range(N):
    psi[i] = psi_exact(i * dx, 0)

# Inicialización de la función de onda exacta, normalizada
psi_exact_init = psi_exact(np.linspace(0, L, N), 0)

# Normalizamos de nuevo por las dudas...
psi = psi/np.sqrt(dx * np.dot(psi.conjugate(),psi))

# Aplicamos el término de energía potencial. Es diagonal, así que equivale a multiplicar por una fase
def aplica_medioV(fdo, V):
    for j in range(N):
        fdo[j] = fdo[j] * np.exp(-1.0j * dt * V[j] / 2.0)
    return fdo

# para almacenar la función de onda a diferentes tiempos y porder hacer una animación
evolution = []

# Para almacenar la función de onda exacta a diferentes tiempos
evolution_exact = []

# evolución de la función de onda
for t in range(Nt):

    psi = aplica_medioV(psi, V)

    """ Aplicamos la parte de energía cinética usando la descomposición LU"""
    # En el borde
    bi[0] = B * psi[0] + psi[1]
    bi[N - 1] = psi[N - 2] + B * psi[N - 1]
    # en el resto de los puntos
    for j in range(1, N - 1):
        bi[j] = psi[j - 1] + B * psi[j] + psi[j + 1]

    # propagación hacia adelante
    aux[0] = uop[0] * bi[0]
    for j in range(1, N):
        aux[j] = uop[j] * (bi[j] + aux[j - 1])

    # propagación hacia atrás
    psi[N - 1] = aux[N - 1]
    for j in reversed(range(N-1)):
      psi[j] = aux[j] + uop[j] * psi[j+1]

    psi = aplica_medioV(psi,V)

    #psi = psi/np.sqrt(dx * np.dot(psi.conjugate(),psi))

    #saltea algunos tiempos
    if(t%skip == 0):
      evolution.append(np.abs(psi)**2)
      evolution_exact.append(np.abs(psi_exact(np.linspace(0, L, N), t * dt))**2)

# El resto es para hacer la animación
fig = plt.figure(figsize=(8,6))
ax = plt.axes(xlim=(0, L), ylim=(0, 0.4))
line, = ax.plot([], [], lw=2)

# primer frame
def init():
    line.set_data([], [])
    return line,

# Cada cuadro corresponde a un tiempo
def aframe(i):
    ax.clear()
    ax.set_xlim(0, L)
    ax.set_ylim(0, 0.4)
    x = np.linspace(0, L, N)
    y = evolution[i]
    ax.set_xlabel('x')
    ax.set_ylabel('$|\\psi|^2$')
    y_exact = evolution_exact[i]
    line, = ax.plot(x, y, lw=2, label = "Crank-Nicolson")
    ax.plot(x, y_exact, color='red', linestyle='dashed', label="Analítica libre")
    ax.plot(x,V/20, color='green', label="potencial reescaleado")
    ax.legend()
    return line,

# Animación.  Con blit=True solo reescribe lo que cambia
anim = animation.FuncAnimation(fig, aframe, init_func=init,
                               frames=cuadros, interval=100, blit=True)

#cierra la figura para que no haga un plot estático abajo de la animación
plt.close(fig)
#animación
HTML(anim.to_jshtml())



Output hidden; open in https://colab.research.google.com to view.

###Estado coherente

Vamos a ver la evolución temporal de un [estado coherente](https://en.wikipedia.org/wiki/Coherent_state) en el oscilador armónico cuántico.

$$
\mathcal{H} = \frac{{\hat p}^2}{2m} + \frac{1}{2} m \omega^2 {\hat x}^2
$$

### Función de onda de un estado coherente
Un estado coherente es un autoestado del operador de destrucción
$$a|\alpha\rangle=\alpha|\alpha\rangle$$
donde $\alpha$ es en general un número complejo.

En la representación de coordenadas corresponde a
$$\sqrt{\frac{m \omega}{2 \hbar}}\left(x+\frac{\hbar}{m\omega}\frac{\partial }{\partial x}\right)\psi_\alpha(x,0)=\alpha\psi_\alpha(x,0), $$
que tiene solución
$$\psi_{\alpha}(x,0)=\left(\frac{m\omega}{\pi\hbar}\right)^{1/4} \exp \Bigg( -\frac{m\omega}{2\hbar}\left(x-\sqrt{\frac{2\hbar}{m\omega}}\Re[\alpha]\right)^2+i\sqrt{\frac{2m\omega}{\hbar}}\Im[\alpha]x\Bigg) ~ $$




Si inicializamos la función de onda en un estado coherente, se tiene a tiempo $t$ que:
$$
   \langle \hat{p}(t) \rangle = |\alpha|\sqrt{2m\hbar\omega} \sin (\arg(\alpha) - \omega t)
$$
y
$$
   \langle \hat{x}(t) \rangle = \sqrt{\frac{2\hbar}{m\omega}}\Re[\alpha(t)]= |\alpha|  \sqrt{\frac{2\hbar}{m\omega}} \cos (\arg(\alpha) - \omega t)~,  
$$

La densidad de probabilidad es una gaussiana centrada en la media que oscila,
$$
|\psi^{(\alpha)}(x,t)|^2=\sqrt{\frac{m\omega}{\pi\hbar} } e^{-\frac{m\omega}{\hbar}\left(x-  \langle \hat{x}(t) \rangle  \right)^2  }  
$$







In [ ]:
# @title
# Método de Crank-Nicolson
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
from IPython.display import HTML
import sys



# masa
m = 1.0 #NO CAMBIAR de 1
# constante del resorte
k = 4.0
# frecuencia angular
omega= np.sqrt(k/m)

# constante de Planck reducida
hbar = 1 # NO CAMBIAR de 1

# Número de puntos
N = 1000
# longitud del sistema
L = 6.0
# paso de discretización
dx = L/(N-1)
# Número de pasos de tiempo
Nt = 100 #int(tmax / dt)
# omite tiempos
omite = 1 #round(Nt/100)
# tiempo total
tmax = 2*np.pi/omega # un período
# paso de tiempo
dt = tmax/Nt

# autovalor de 'a' del estado coherente
alpha  = complex(L/2.0 + 0.5j)

# potencial externo
def potencial(x):
    return 0.5*m*omega**2*(x-L/2.0)*(x-L/2.0)

# función de onda
psi = np.zeros(N, dtype=complex)
aux = np.zeros(N, dtype=complex)

# Parámetros auxiliares para la inversión del operador de energía cinética
bi = np.zeros(N, dtype=complex)
uop = np.zeros(N, dtype=complex)


A = 2.0 - 4j * (dx ** 2) / (dt)
B = -2.0 - 4j * (dx ** 2) / (dt)

uop[0] = 1.0 / A
for i in range(1, N):
    uop[i] = 1.0 / (A - uop[i - 1])

# Definimos el potencial
V = np.zeros(N)
for i in range(1, N):
    V[i] = potencial(i * dx)


# Inicializamos de la función de onda, normalizada
for i in range(N):
    psi[i] = np.sqrt(np.sqrt(m*omega/(hbar*np.pi)))* np.exp(- ((m*omega)/(2*hbar))*(i * dx - np.sqrt(2*hbar/(m*omega))*alpha.real) ** 2) * np.exp(1j * np.sqrt(2*m*omega/hbar) * alpha.imag*i * dx)

# Normalizamos de nuevo por las dudas...
psi = psi/np.sqrt(dx * np.dot(psi.conjugate(),psi))

# Aplicamo el término de energía potencial. Es diagonal, así que equivale a multiplicar por una fase
def aplica_medioV(fdo, V):
    for j in range(N):
        fdo[j] = fdo[j] * np.exp(-1.0j * V[j] * dt / 2.0)
    return fdo

# para almacenar la función de onda a diferentes tiempos y porder hacer una animación
evolution = []

# evolución de la función de onda
for t in range(Nt):

    psi = aplica_medioV(psi, V)

    """ Aplicamos la parte de energía cinética usando la descomposición LU"""
    # En el borde
    bi[0] = B * psi[0] + psi[1]
    bi[N - 1] = psi[N - 2] + B * psi[N - 1]
    # en el resto de los puntos
    for j in range(1, N - 1):
        bi[j] = psi[j - 1] + B * psi[j] + psi[j + 1]

    # propagación hacia adelante
    aux[0] = uop[0] * bi[0]
    for j in range(1, N):
        aux[j] = uop[j] * (bi[j] + aux[j - 1])

    # propagación hacia atrás
    psi[N - 1] = aux[N - 1]
    for j in reversed(range(N-1)):
      psi[j] = aux[j] + uop[j] * psi[j+1]

    psi = aplica_medioV(psi,V)

    #psi = psi/np.sqrt(dx * np.dot(psi.conjugate(),psi))

    #saltea algunos tiempos
    if(t%omite == 0):
      evolution.append(np.abs(psi)**2)


# El resto es para hacer la animación
fig = plt.figure(figsize=(8,6))
ax = plt.axes(xlim=(0, L), ylim=(0, 3))
line, = ax.plot([], [], lw=2)

# primer frame
def init():
    line.set_data([], [])
    return line,

# Cada cuadro corresponde a un tiempo
def aframe(i):
    x = np.linspace(0, L, N)
    y = evolution[i]
    ax.set_xlabel('x')
    ax.set_ylabel('$|\\psi|^2$')
    line.set_data(x, y)

    return line,

# Animación.  Con blit=True solo reescribe lo que cambió
anim = animation.FuncAnimation(fig, aframe, init_func=init,
                               frames=100, interval=100, blit=True)

#cierra la figura para que no haga un plot estático abajo de la animación
plt.close(fig)
#animación
HTML(anim.to_jshtml())
